# Imports

In [9]:
%matplotlib inline
from numpy import *

# Simulation code

In [47]:
def init_config(N,strategy):
    """Initializes a configurations.
    
    strategy -  one of random / all-up / all-down
    """
    if strategy == "random":
        return 2*random.randint(0,2,N)-1
    elif strategy == "all-up":
        return ones((N,),dtype=int)
    elif strategy == "all-down":
        return -1*ones((N,),dtype=int)
    else:
        raise ValueError("Unknown strategy in init_config")

In [ ]:
ASSERT_EPS = 1.0e-7

class IsingModelMC_1D(object):
    "Implementation of a Metropolis Monte Carlo simulation of the 1d Ising model"
    
    def __init__(self,pars):
        """
        Constructor - initializes simulation
        
        pars - dictionary that contains parameters of the simulation. The following keys
               are required:
               -) system_size (int > 0)
               -) init_strategy (random / all-up / all-down)
               -) boundary_conditions (free / pbc)
               -) h_over_j (float)
               -) T (float > 0)
        """
        # PARAMETERS
        self.N = int(pars["system_size"])
        if self.N <= 0:
            raise ValueError("system_size must be larger than 0")
        
        if pars["boundary_conditions"] == "pbc":
            self.pbc = True
        elif pars["boundary_conditions"] == "free":
            self.pbc = False
        else:
            raise ValueError("Unknown boundary conditions")
        
        self.h_over_j = pars["h_over_j"]
        self.T = float(pars["T"])        
        if self.T <= 0.0:
            raise ValueError("T must be larger than 0")
        self.beta = 1.0/pars["T"]
        
        # SIMULATION VARIABLES
        self.config = init_config(self.N,pars["init_strategy"])
        
        # STATISTICAL VARIABLES
        self.reset_statistics_variables()
            
    def reset_statistics_variables(self):
        "Refreshes the values of all variables that are cumulatively updated while the simulation runs"
        self.n_attempted = 0
        self.n_accepted = 0
        
        self.t_timeseries = []
        
        self.E = self.get_E()
        self.E_sum = 0.0
        self.E_sum_samples = 0
        self.E_timeseries = []
        
        self.m = self.get_m()
        self.m_sum = 0.0
        self.m_sum_samples = 0.0
        self.m_timeseries = []
        self.m_increment = 2.0/self.N
        
    def __str__(self):
        "Returns a string that contains the current simulation state"
        s = "****************************************************************"
        s += "\n** Ising model MC simulation"
        s += "\n- % 10s = %i"%("N",self.N)
        s += "\n- % 10s = %s"%("PBC","YES" if self.pbc else "NO")
        s += "\n- % 10s = %.2f"%("h_over_j",self.h_over_j)
        s += "\n- % 10s = %.2f"%("E",self.E)
        s += "\n- % 10s = %.2f"%("T",self.T)
        s += "\n- % 10s = %.2f"%("beta",self.beta)
        s += "\n-- STATISTICS --------------------------------------------------"
        s += "\n- % 10s = %.2f"%("E",self.E)
        s += "\n- % 10s = %.2f"%("m",self.m)
        s += "\n- % 10s = %i"%("n_attempted",self.n_attempted)
        s += "\n- % 10s = %i"%("n_accepted",self.n_accepted)
        s += "\n- % 10s = %.2f"%("acc_ratio",float(self.n_accepted)/float(self.n_attempted) if self.n_attempted > 0 else 0.0)
        s += "\n- CONFIG:"
        s += "\n//%s//"%self.config_to_s()
        s += "\n****************************************************************"
        return s
        
    def config_to_s(self):
        "Returns a string that represents the current configuration"
        s = "%s"%('+' if self.config[0] == 1 else '-')
        for i in range(1,self.N):
            s += (" +" if self.config[i] == 1 else " -")
        return s
    
    def reset_cumulative_variables(self):
        "Update all variables that are cumulatively tracked while the simulation is running"
        self.m = self.get_m()
        self.E = self.get_E()

    def print(self):
        "Print the current simulation state"
        print(str(self))

    def get_E_single(self,i):
        """Return the contribution of spin i to the total energy
        
        Pairwise energies are split evenly between the two particles involved"""
        nbs = self.get_neighbors(i)
        sum_E = 0.0
        me = self.config[i]
        for j in nbs:
            sum_E += -0.5*me*self.config[j]
        sum_E += -self.h_over_j*me
        return sum_E
    
    def get_m(self):
        "Return the current magnetization"
        sum_m = 0
        for i in range(self.N):
            sum_m += self.config[i]
        return sum_m/self.N
        
    def do_sweep(self,debug_level=0):
        "Perform one simulation sweep that consists of (number of particles) MC moves"
        for i in range(self.N):
            if debug_level >= 2:
                print("* Doing move % 5i / % 5i"%(i+1,self.N))
            self.do_move(debug_level=debug_level)
            
    def accept_move(self,i,dE,debug_level=0):
        """Accept a flip of spin i
        
        If debug_level >= 3 or higher a consistency check of the cumulative variables E
        and m is performed. If debug_level >= 4 the state of the system before the move 
        is accepted is printed.
        """
        if debug_level >= 4:
            self.print()
        self.n_accepted += 1
        self.m -= self.m_increment*self.config[i]
        self.config[i] *= -1
        self.E += dE
        if debug_level >= 3:
            assert fabs(self.m - self.get_m()) < ASSERT_EPS, "inconsistent magnetization: running m = %.8f != %.8f = summed m"%(self.m,self.get_m())
            assert fabs(self.E - self.get_E()) < ASSERT_EPS, "inconsistent energy: running E = %.8f != %.8f = summed E"%(self.E,self.get_E())

        
    def do_stats(self,i,debug_level=0):
        "Update the statistics based on the current state of the system"
        self.t_timeseries.append(i)
        
        self.E_sum += self.E
        self.E_sum_samples += 1
        self.E_timeseries.append(self.E)
        
        # There used to be a bug here (extra division by N). m_sum is the
        # sum over all observations, so for the actual average, you have
        # to divide by the number of observations later.
        self.m_sum += self.m    
        self.m_sum_samples += 1
        self.m_timeseries.append(self.m)
        
    def print_stats_header(self):
        "Print a header line for the statistics output"
        s = "% 10s"%"t"
        s += " | % 10s"%"MAGN"
        s += " | % 10s"%"AVG MAGN"
        s += " | % 10s"%"ENERGY"
        s += " | % 10s"%"AVG ENERGY"
        s += " | % 10s"%"ACC RATIO"
        print(s)
        
    def print_stats(self,i):
        "Print the statistics output"
        s = "% 10s"%("% 8i"%i) # timestep
        
        s += " | % 10s"%("% 8.2f"%self.m)
        avg_m = self.m_sum / self.m_sum_samples if self.m_sum_samples > 0 else 0.0
        s += " | % 10s"%("% 8.2f"%avg_m)
        
        s += " | % 10s"%("% 8.2f"%self.E)
        avg_E = self.E_sum / self.E_sum_samples if self.E_sum_samples > 0 else 0.0
        s += " | % 10s"%("% 8.2f"%avg_E)
        
        acc_ratio = float(self.n_accepted)/float(self.n_attempted) if self.n_attempted > 0 else 0.0
        s += " | % 10s"%("% 8.2f"%acc_ratio)
        print(s)
        
    def run(self,n_sweeps,pars,debug_level=0):
        """Run a simulation
        
        n_sweeps - int: Number of sweeps to be performed
        pars - dict: parameters used for the simulation. Must include keys i_stats and i_stats_out
        debug_level - int: 0 or larger, controls level of debug output
        
        """
        i_stats = int(pars["i_stats"])
        i_stats_out = int(pars["i_stats_out"]) if i_stats > 0 else 0
        if i_stats_out > 0: 
            self.print_stats_header()
        for i in range(n_sweeps):
            self.do_move(debug_level=debug_level)
            if i_stats > 0 and (i % i_stats == 0):
                self.do_stats(i)
            if i_stats_out > 0 and (i % i_stats_out == 0):
                self.print_stats(i)
            if i % 10000 == 0:
                self.reset_cumulative_variables()
                
    def get_timeseries(self):
        """Return timeseries tracked while the last simulation was run
        
        Returns a dictionary with the name of variable as key and numpy.ndarray as values
        """
        return dict(
            t = array(self.t_timeseries),
            m = array(self.m_timeseries),
            E = array(self.E_timeseries)
        )
    
    ##########################################################################################################
    ## METHODS TO IMPLEMENT                                                                                 ##
    ##########################################################################################################    
            
    def get_neighbors(self,i):
        "Return the neighbors of spin i as an iterable (list or tuple)"
        ## -- START IMPLEMENTING HERE

        if i < 0 or i > self.N -1:
            raise ValueError("index outside of bounds.")

        index = i
        
        neighbors = array([index-1,index+1])
        check_pbc = self.pbc | ((neighbors >= 0) & (neighbors < self.N))

        return (neighbors[check_pbc] % self.N).tolist()


        ## -- STOP IMPLEMENTING HERE
        return neighbors
    
    def get_E(self):
        "Return the total current total energy of the system"
        E = 0.0
        ## -- START IMPLEMENTING HERE
        # Calculate total potential energy of the system
        # Use the get_neighbors function you implemented above and self.get_E_single
        for i in range(self.N):
            E += self.get_E_single(i)
        
        ## -- STOP IMPLEMENTING HERE
        
        return E
    
    def get_dE_single(self,i):
        "Return the change in energy of particle i is flipped"
        dE = 0.0
        ## -- START IMPLEMENTING HERE
        # Calculate the change in potential energy if spin i were to be flipped
        # Use the get_neighbors function you implemented above

        neighbors = self.get_neighbors(i)

        current_spin = self.config[i]
        new_spin = -1*current_spin
        spin_change = new_spin - current_spin

        # pair spin change
        for neighbor in neighbors:
            dE -= spin_change * self.config[neighbor]

        # Magnetic field spin change
        dE += -self.h_over_j * spin_change
        
        ## -- STOP IMPLEMENTING HERE
        return dE
    
    def acceptance(self,dE,debug_level=0):
        "Return true if Metropolis MC acceptance criterion is accepted - false otherwise"
        ## -- START IMPLEMENTING HERE
        # Implement the Metropolis acceptance criterion. dE is the difference in potential energy
        
        if dE <= 0 or random.rand() < exp(-dE*(self.T+1e-12)**(-1)):
            return True
        else:
            return False

        ## -- STOP IMPLEMENTING HERE
    
    def do_move(self,debug_level=0):
        "Perform a single Metropolis MC move including an acceptance test"
        accept = False
        ## -- START IMPLEMENTING HERE
        i = random.choice(self.N)
        dE = self.get_dE_single(i)

        accept = self.acceptance(dE, debug_level=debug_level)
        ## -- STOP IMPLEMENTING HERE
        
        self.n_attempted += 1
        if accept:
            self.accept_move(i,dE,debug_level=debug_level)
        else:
            pass

# Initialization

In [183]:
pars = dict(
    system_size = 1000,
    h_over_j = 0.0,
    init_strategy = "all-up",
    boundary_conditions = "pbc",
    T=0.2,
    i_stats = 10,
    i_stats_out = 10000
)

system = IsingModelMC_1D(pars)
system.print()

****************************************************************
** Ising model MC simulation
-          N = 1000
-        PBC = YES
-   h_over_j = 0.00
-          E = -1000.00
-          T = 0.20
-       beta = 5.00
-- STATISTICS --------------------------------------------------
-          E = -1000.00
-          m = 1.00
- n_attempted = 0
- n_accepted = 0
-  acc_ratio = 0.00
- CONFIG:
//+ + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + +

# Run

In [184]:
system.run(100000,pars,debug_level=0)
system.print()

         t |       MAGN |   AVG MAGN |     ENERGY | AVG ENERGY |  ACC RATIO
         0 |       1.00 |       1.00 |   -1000.00 |   -1000.00 |       0.00
     10000 |       1.00 |       1.00 |   -1000.00 |   -1000.00 |       0.00
     20000 |       1.00 |       1.00 |   -1000.00 |   -1000.00 |       0.00
     30000 |       1.00 |       1.00 |   -1000.00 |   -1000.00 |       0.00
     40000 |       1.00 |       1.00 |   -1000.00 |   -1000.00 |       0.00
     50000 |       1.00 |       1.00 |   -1000.00 |   -1000.00 |       0.00
     60000 |       1.00 |       1.00 |   -1000.00 |   -1000.00 |       0.00
     70000 |       1.00 |       1.00 |   -1000.00 |   -1000.00 |       0.00
     80000 |       1.00 |       1.00 |   -1000.00 |   -1000.00 |       0.00
     90000 |       1.00 |       1.00 |   -1000.00 |   -1000.00 |       0.00
****************************************************************
** Ising model MC simulation
-          N = 1000
-        PBC = YES
-   h_over_j = 0.00
-          